In [10]:
import sys
import os
import numpy as np
import collections
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords 
from nltk.tokenize import RegexpTokenizer, sent_tokenize, word_tokenize

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM,Dropout,Dense,Embedding,Flatten
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.callbacks import ModelCheckpoint

train_data_size = 1000
test_data_size = 1000
DIC_SIZE = 10000
MAX_LEN = 1575
# nltk.download('stopwords')

In [4]:
# dic = buildVocabulary(os.getcwd()+'/aclImdb/train/')
vocabulary = loadDic(os.getcwd()+'/aclImdb/imdb.vocab')
Xtrain, Xtest, ytrain, ytest = loadData(os.getcwd()+'/aclImdb/', vocabulary) # tokens


In [59]:
vocabulary = loadDic(os.getcwd()+'/aclImdb/imdb.vocab')


# load tokens
trn_tokens, test_tokens, trn_labels, test_labels = loadData(os.getcwd()+'/aclImdb/', vocabulary)



## Method1: Logistic Regression using word-embeddings sentence representation
# load pre-trained 50D word embedding
word_embedding = load_word_embedding('glove.6B/glove.6B.200d.txt')


# build sentence representations
trn_representations = construct_sentence_rep(trn_tokens, word_embedding)
test_representations = construct_sentence_rep(test_tokens, word_embedding)


# Logistic Regression Text Classification using sentence representations
logestic_regression(trn_representations, trn_labels, test_representations, test_labels)

NameError: name 'dev_representations' is not defined

In [170]:
# model = LSTM_model()
# model.summary()
# model.fit(trn_index[:1],trn_labels[:1])
# dic = buildVocabulary(os.getcwd()+'/aclImdb/train/')
trn_text, test_text, trn_labels, test_labels = loadText(os.getcwd()+'/aclImdb/')
tokenizer = Tokenizer(num_words=100000)
tokenizer.fit_on_texts(list(trn_text)+list(test_text))
X_train = tokenizer.texts_to_sequences(trn_text)
X_test = tokenizer.texts_to_sequences(test_text)
X_train = sequence.pad_sequences(X_train,maxlen=MAX_LEN)
X_test = sequence.pad_sequences(X_test,maxlen=MAX_LEN)

In [25]:
## Method2: LSTM sequence sentenment classifier
trn_text, test_text, trn_labels, test_labels = loadText(os.getcwd()+'/aclImdb/')
tokenizer = Tokenizer(num_words=100000)
tokenizer.fit_on_texts(list(trn_text)+list(test_text))
X_train = tokenizer.texts_to_sequences(trn_text)
X_test = tokenizer.texts_to_sequences(test_text)
X_train = sequence.pad_sequences(X_train,maxlen=MAX_LEN)
X_test = sequence.pad_sequences(X_test,maxlen=MAX_LEN)

model = LSTM_model()
model.summary()


filepath= os.getcwd()+"/weights/lstm.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

model.fit(np.array(X_train[:1000]),trn_labels[:1000], validation_split=0.33, epochs=30, batch_size=64, callbacks=callbacks_list)
# model.fit(np.array(trn_index[:1000]),trn_labels[:1000], validation_split=0.33, epochs=30, batch_size=64, callbacks=callbacks_list)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 300)         30000000  
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               219648    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 30,219,777
Trainable params: 30,219,777
Non-trainable params: 0
_________________________________________________________________
Train on 669 samples, validate on 331 samples
Epoch 1/30
640/669 [===========================>..] - ETA: 1s - loss: 1.3616 - accuracy: 0.5281
Epoch 00001: val_accuracy improved from -inf to 0.58610, saving model to /Users/liuyining/Documents/GitZone/Data Mining/IMDB-Review-Sentiment-Analysis/weights/lstm-01-0.59.hdf5
669/669 [==============================] - 45s 67ms/s

KeyError: 'val_accuracy'

In [21]:
# Method3: Multivariate Bernoulli Naive Bayes Classifier

from sklearn.naive_bayes import MultinomialNB
import math

# dic = buildVocabulary(os.getcwd()+'/aclImdb/train/')
vocabulary = loadDic(os.getcwd()+'/aclImdb/imdb.vocab')
Xtrain, Xtest, ytrain, ytest = loadDataBOW(os.getcwd()+'/aclImdb/', vocabulary) # load BOW representation

# thetaPosTrue, thetaNegTrue = naiveBayesBernFeature_train(Xtrain[:1000], ytrain[:1000])
# yPredict, Accuracy = naiveBayesBernFeature_test(Xtest[:1000], ytest[:1000], thetaPosTrue, thetaNegTrue)
# print("Multivariate Bernoulli Naive Bayes Classifier Acc: ",Accuracy)

Accuracy = naiveBayesMultinomial_MNBC(Xtrain, ytrain, Xtest, ytest)
print("Naive Bayes Classifier Acc accuracy =", Accuracy)

Naive Bayes Classifier Acc accuracy = 1.0


In [20]:

def naiveBayesMultinomial_MNBC(Xtrain, ytrain, Xtest, ytest):
    clf = MultinomialNB()
    clf.fit(Xtrain, ytrain)
    yPredict = clf.predict(Xtest)
    errors = np.sum(np.absolute(yPredict - ytest))
    Accuracy = (ytest.shape[0] - errors)/ytest.shape[0]
    return Accuracy

def naiveBayesBernFeature_train(Xtrain, ytrain):
    thetaPosTrue = np.zeros((Xtrain.shape[1],), dtype = float)
    thetaNegTrue = np.zeros((Xtrain.shape[1],), dtype = float)
    for i in range(len(Xtrain)):
        Binary = np.where(Xtrain[i] > 0, 1, 0)
        if ytrain[i] == 1:
            thetaPosTrue += Binary
        elif ytrain[i] == 0:
            thetaNegTrue += Binary
    thetaPosTrue += 1
    thetaNegTrue += 1
    thetaPosTrue /= (Xtrain.shape[0]/2 + 2)
    thetaNegTrue /= (Xtrain.shape[0]/2 + 2)
    return thetaPosTrue, thetaNegTrue


def naiveBayesBernFeature_test(Xtest, ytest, thetaPosTrue, thetaNegTrue):
    yPredict = []
    for X in Xtest:
        N = np.sum(X) #total words of X
        thetaLogPos = 0.0
        thetaLogNeg = 0.0
        for i in range(len(X)):
            if X[i] > 0:
                thetaLogPos += math.log2(thetaPosTrue[i])
                thetaLogNeg += math.log2(thetaNegTrue[i])
            elif X[i] ==0:
                thetaLogPos += math.log2(1-thetaPosTrue[i])
                thetaLogNeg += math.log2(1-thetaNegTrue[i])
        if thetaLogPos > thetaLogNeg:
            yPredict.append(1)
        else:
            yPredict.append(0)
    yPredict = np.array(yPredict)
#     print(yPredict, ytest)
    errors = np.sum(np.absolute(yPredict - ytest))
    Accuracy = (ytest.shape[0] - errors)/ytest.shape[0]
    return yPredict, Accuracy

In [24]:
# get word index representation
# trn_index = [token_to_index(tokens, vocabulary) for tokens in trn_tokens]
# test_index = [token_to_index(tokens, vocabulary) for tokens in test_tokens]

# # padding word index representation
# trn_index = sequence.pad_sequences(trn_index,maxlen=MAX_LEN)
# test_index = sequence.pad_sequences(test_index,maxlen=MAX_LEN)

# tokenizer = Tokenizer(num_words=100000)
# tokenizer.fit_on_texts(list(trn_tokens)+list(test_tokens))
# X_test = tokenizer.texts_to_sequences(X_test)

NameError: name 'trn_tokens' is not defined

In [22]:
def buildVocabulary(Path):
    tokenizer = RegexpTokenizer(r'\w+') #tokenizer removing puncturations
    ps = PorterStemmer()
    stop_words = set(stopwords.words('english')) 
    tokens = []
    for folder in ['neg','pos']:
        for filename in os.listdir(Path+folder):
            data = open(Path+folder+'/'+filename, 'r')
            raw = data.read()
            token = tokenizer.tokenize(raw)
            # stop words removal
            filtered_token = [w for w in token if not w in stop_words]
#             filtered_token = [ps.stem(w) for w in token if not w in stop_words]
            # stemming
            for i in range(len(filtered_token)):
                filtered_token[i] = ps.stem(filtered_token[i])
            tokens.extend(filtered_token)
    word_counts = collections.Counter(tokens).most_common(DIC_SIZE-1)
    vocab = {}
    i = 0
    for word_count in word_counts:
        vocab[word_count[0]] = i
        i += 1
    vocab['UNK'] = i
    return vocab

def loadDic(Path, skip_stop = False, stem = False, lower = True):
    dic = open(Path,'r')
    dic = dic.read().split('\n')
    
    # stop words removal and stemming
    ps = PorterStemmer()
    stop_words = set(stopwords.words('english'))
    if stem:
        dic = list(set([ps.stem(w) for w in dic]))
    if skip_stop:
        dic = list(set([w for w in dic if not w in stop_words]))
    if lower:
        dic = list(set([w.lower() for w in dic]))
    else:
        dic = list(set(dic))
    
    
    # transfer to dictionary
    
    filtered_dic = {word: i for i, word in enumerate(dic)}
    filtered_dic['UNK'] = len(filtered_dic)
    return filtered_dic

def loadDataBOW(Path, vocabulary):
    voc_size = len(vocabulary)
    Xtrain = np.zeros((train_data_size*2,voc_size), dtype=int)
    Xtest = np.zeros((train_data_size*2,voc_size), dtype=int)
    ytrain = np.concatenate((np.ones(train_data_size, dtype=int),np.zeros(train_data_size, dtype=int)), axis = 0)
    ytest = np.concatenate((np.ones(train_data_size, dtype=int),np.zeros(train_data_size, dtype=int)), axis = 0)
    
    for i, filename in enumerate(os.listdir(Path+'train/pos/')[:train_data_size]):
#         Xtrain.append(transfer(Path+'train/pos/'+filename, vocabulary))
        Xtrain[i] = transfer(Path+'train/pos/'+filename, vocabulary)

    for i, filename in enumerate(os.listdir(Path+'train/neg/')[:train_data_size]):
        Xtrain[i] = transfer(Path+'train/neg/'+filename, vocabulary)
#         Xtrain.append(transfer(Path+'train/neg/'+filename, vocabulary))

    for i, filename in enumerate(os.listdir(Path+'test/pos/')[:test_data_size]):
        Xtest[i] = transfer(Path+'test/pos/'+filename, vocabulary)
#         Xtest.append(transfer(Path+'test/pos/'+filename, vocabulary))

    for i, filename in enumerate(os.listdir(Path+'test/neg/')[:test_data_size]):
        Xtest[i] = transfer(Path+'test/neg/'+filename, vocabulary)
#         Xtest.append(transfer(Path+'test/neg/'+filename, vocabulary))

#     Xtrain, Xtest = np.array(Xtrain), np.array(Xtest)
    Xtrain, ytrain = shuffle(Xtrain, ytrain, random_state=0)
    Xtest, ytest = shuffle(Xtest, ytest, random_state=0)
    return Xtrain, Xtest, ytrain, ytest


def loadText(Path):
    Xtrain = []
    Xtest = []
    ytrain = np.concatenate((np.ones(train_data_size, dtype=int),np.zeros(train_data_size, dtype=int)), axis = 0)
    ytest = np.concatenate((np.ones(train_data_size, dtype=int),np.zeros(train_data_size, dtype=int)), axis = 0)
    
    for i, filename in enumerate(os.listdir(Path+'train/pos/')[:train_data_size]):
        data = open(Path+'train/pos/'+filename,'r')
        raw = data.read().lower()
        Xtrain.append(raw)
#         max_len = max(len(tokens),max_len)

    for i, filename in enumerate(os.listdir(Path+'train/neg/')[:train_data_size]):
        data = open(Path+'train/neg/'+filename,'r')
        raw = data.read()
        Xtrain.append(raw)
#         max_len = max(len(tokens),max_len)

    for i, filename in enumerate(os.listdir(Path+'test/pos/')[:test_data_size]):
        data = open(Path+'test/pos/'+filename,'r')
        raw = data.read()
        Xtest.append(raw)

    for i, filename in enumerate(os.listdir(Path+'test/neg/')[:test_data_size]):
        data = open(Path+'test/neg/'+filename,'r')
        raw = data.read()
        Xtest.append(raw)
        
#     print(max_len)

    Xtrain, ytrain = shuffle(Xtrain, ytrain, random_state=0)
    Xtest, ytest = shuffle(Xtest, ytest, random_state=0)
    return Xtrain, Xtest, ytrain, ytest

def loadData(Path, vocabulary):
    Xtrain = []
    Xtest = []
    ytrain = np.concatenate((np.ones(train_data_size, dtype=int),np.zeros(train_data_size, dtype=int)), axis = 0)
    ytest = np.concatenate((np.ones(train_data_size, dtype=int),np.zeros(train_data_size, dtype=int)), axis = 0)
    
    tokenizer = RegexpTokenizer(r'\w+') #tokenizer removing puncturations
#     max_len = 0
    for i, filename in enumerate(os.listdir(Path+'train/pos/')[:train_data_size]):
        data = open(Path+'train/pos/'+filename,'r')
        raw = data.read().lower()
        tokens = tokenizer.tokenize(raw)
        Xtrain.append(tokens)
#         max_len = max(len(tokens),max_len)

    for i, filename in enumerate(os.listdir(Path+'train/neg/')[:train_data_size]):
        data = open(Path+'train/neg/'+filename,'r')
        raw = data.read()
        tokens = tokenizer.tokenize(raw)
        Xtrain.append(tokens)
#         max_len = max(len(tokens),max_len)

    for i, filename in enumerate(os.listdir(Path+'test/pos/')[:test_data_size]):
        data = open(Path+'test/pos/'+filename,'r')
        raw = data.read()
        tokens = tokenizer.tokenize(raw)
        Xtest.append(tokens)

    for i, filename in enumerate(os.listdir(Path+'test/neg/')[:test_data_size]):
        data = open(Path+'test/neg/'+filename,'r')
        raw = data.read()
        tokens = tokenizer.tokenize(raw)
        Xtest.append(tokens)
        
#     print(max_len)

    Xtrain, ytrain = shuffle(Xtrain, ytrain, random_state=0)
    Xtest, ytest = shuffle(Xtest, ytest, random_state=0)
    return Xtrain, Xtest, ytrain, ytest


def transfer(fileDj, vocabulary):
    data = open(fileDj,'r')
    BOWDj = np.zeros((len(vocabulary),), dtype = int)
    
    tokenizer = RegexpTokenizer(r'\w+') #tokenizer removing puncturations
    ps = PorterStemmer()
    stop_words = set(stopwords.words('english')) 
    
    raw = data.read()
    tokens = tokenizer.tokenize(raw)
    # stop words removal
    filtered_token = [ps.stem(w) for w in tokens if not w in stop_words]
#     # stemming
#     for i in range(len(filtered_token)):
#         filtered_token[i] = ps.stem(filtered_token[i])
    # transfer to BOW
    for token in filtered_token:
        if token in vocabulary:
            BOWDj[vocabulary[token]] += 1
        else:
            BOWDj[vocabulary['UNK']] += 1
    
    return BOWDj


def token_to_index(tokens, vocabulary):
    lower_token = [w.lower() for w in tokens]
    index = [vocabulary.get(token, vocabulary['UNK']) for token in lower_token]
    #add <stop> to the last position
    index.append(len(vocabulary)+1)
    #add <start> to the first position
    index1 = [len(vocabulary)]
    index1.extend(index)
    return index1


def loadWordIndex(Path, vocabulary):
    voc_size = len(vocabulary)
    Xtrain = []
    Xtest = []
    ytrain = np.concatenate((np.ones(train_data_size, dtype=int),np.zeros(train_data_size, dtype=int)), axis = 0)
    ytest = np.concatenate((np.ones(train_data_size, dtype=int),np.zeros(train_data_size, dtype=int)), axis = 0)
    
    tokenizer = RegexpTokenizer(r'\w+') #tokenizer removing puncturations
    ps = PorterStemmer()
    stop_words = set(stopwords.words('english')) 

    for i, filename in enumerate(os.listdir(Path+'train/pos/')[:train_data_size]):
        data = open(Path+'train/pos/'+filename,'r')
        raw = data.read()
        tokens = tokenizer.tokenize(raw)
        Xtrain.append(token_to_index(tokens, vocabulary))

    for i, filename in enumerate(os.listdir(Path+'train/neg/')[:train_data_size]):
        data = open(Path+'train/neg/'+filename,'r')
        raw = data.read()
        tokens = tokenizer.tokenize(raw)
        Xtrain.append(token_to_index(tokens, vocabulary))

    for i, filename in enumerate(os.listdir(Path+'test/pos/')[:test_data_size]):
        data = open(Path+'test/pos/'+filename,'r')
        raw = data.read()
        tokens = tokenizer.tokenize(raw)
        Xtest.append(token_to_index(tokens, vocabulary))

    for i, filename in enumerate(os.listdir(Path+'test/neg/')[:test_data_size]):
        data = open(Path+'test/neg/'+filename,'r')
        raw = data.read()
        tokens = tokenizer.tokenize(raw)
        Xtest.append(token_to_index(tokens, vocabulary))

    return Xtrain, Xtest, ytrain, ytest    


def load_word_embedding(Path):
    file = open(Path, 'r')
    embedding = {}
    for line in file:
        line = line.split()
        embedding[line[0]] = np.array(line[1:], dtype = float)
    return embedding


def construct_sentence_rep(data, word_embedding, dim=50):
    sentence_reps = []
    for tokens in data:
        current_res = np.zeros((dim,), dtype=float)
        for token in tokens:
            if token in word_embedding:
                current_res += word_embedding[token]
            else:
                current_res += word_embedding['unk']
        if len(tokens)==0: 
            current_res = word_embedding['unk']
        else:
            current_res /= len(tokens)
        sentence_reps.append(current_res)
    return np.array(sentence_reps, dtype=float)


def logestic_regression(trn_x, trn_labels, dev_x, dev_labels):
    # Define a LR classifier
    classifier = LogisticRegression(random_state=0,tol=0.0001,solver='liblinear',
                                    multi_class='auto',C=0.1, penalty='l1') #0.647
#     classifier = LogisticRegression(random_state=0,tol=0.0002,solver='liblinear',
#                                     multi_class='auto',C=0.1, penalty='l1') 
#     classifier = LogisticRegression(verbose=1, C=5, penalty='l2')
    classifier.fit(trn_x, trn_labels)

    # Measure the performance on dev data
    # print("Training accuracy = %f" % classifier.score(trn_x, trn_labels))
    print("Dev accuracy = ", classifier.score(dev_x, dev_labels))
    
    
def LSTM_model():
    model = Sequential()
    model.add(Embedding(input_dim=100000,output_dim=300))
    model.add(LSTM(units=128,dropout=0.2))
    model.add(Dense(1))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    return model